In [5]:
import torch.nn as nn
import torch

In [6]:
# Realiza el entrenamiento de una epoca
# Argumentos:
#       - model: modelo que queremos entrenar
#       - test_dataloader: lotes de prueba
#       - loss_criterion: define la funcion de perdida
#       - device: cpu o cuda, define si el entrenamiento se hace en la cpu o gpu

def validate_unet(model, test_dataloader, loss_criterion, device):
    
    # Establece el modelo en modo de evaluación
    model.eval()  
    
    # Calcular la precisión
    correct = 0
    total = 0
    
    # No tenemos en cuenta el gradiente
    with torch.no_grad():
        
        # Iteramos sobre los lotes de test
        for images, masks in test_dataloader:
            
            # Permuta las dimensiones de las imágenes a (N, C, H, W)
            images = images.permute(0, 3, 1, 2)

            # Movemos al device las imagenes y las Ground Truth
            images = images.to(device, dtype=torch.float)
            masks = masks.to(device, dtype=torch.long)
            
            # El modelo realiza la inferencia para las imagenes del lote
            outputs = model(images)
            
            # predicted --> tensor que contiene la clase predicha con la mayor probabilidad para cada píxel en la salida del modelo
            _, predicted = torch.max(outputs.data, 1)
            total += masks.nelement()  # Número total de píxeles en la máscara
            
            # eq: compara elemento a elemento la prediccion con los valores reales de la imagen Ground Truth. generando un tensor de booleanos
            # sum: suma los elementos true del tensor, es decir, los aciertos
            correct += predicted.eq(masks.data).sum().item()  

    accuracy = round((correct / total)*100,2)  # Precisión del modelo

    
    return accuracy